In [1]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from faker import Faker
from time import sleep
from confluent_kafka.schema_registry.protobuf import ProtobufSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient
import sys
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer
from confluent_kafka import SerializingProducer
import random

In [2]:
# Load environment variables from .env file
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
kafka_host = os.getenv('KAFKA_HOST')
topic_name = os.getenv('KAFKA_TOPIC_NAME')
replication_factor = int(os.getenv('KAFKA_REPLICATION'))
num_partitions = int(os.getenv('KAFKA_PARTITION'))
schema_registry_host = os.getenv('SCHEMA_REG_HOST')

In [4]:
# Protobuf schema import
sys.path.append('../protobuf')
import schema_pb2

In [5]:
# Kafka Configuration
bootstrap_servers = f'{kafka_host}:9092'
schema_registry_url = f'http://{schema_registry_host}:8081'

In [6]:
# Schema Registry Client
schema_registry_client = SchemaRegistryClient({
    'url': f'http://{os.getenv("SCHEMA_REG_HOST")}:8081'
})

In [ ]:
'schema.registry.url': 'http://schema-registry:8081'


In [7]:
# Protobuf Serializer
protobuf_serializer = ProtobufSerializer(
    schema_pb2.PetShopEvent, 
    schema_registry_client, 
    conf={'use.deprecated.format': False} 
)


In [8]:
# Producer Configuration
producer_conf = {
    'bootstrap.servers': bootstrap_servers,
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer': protobuf_serializer  # Use Protobuf serializer for values
}

In [9]:
# Initialize Kafka producer
producer = SerializingProducer(producer_conf)

In [10]:
# Define callback function to handle the delivery report
def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}] @ offset {msg.offset()}")

In [11]:
# Initialize Faker for fake data generation
fake = Faker()

In [12]:

def generate_pet_shop_event():
    """Generate a fake PetShopEvent using Faker and Protobuf."""
    pet_event = schema_pb2.PetShopEvent()
    pet_event.pet_id = str(uuid.uuid4())
    pet_event.pet_name = fake.first_name()
    pet_event.species = fake.random_element(elements=('Dog', 'Cat', 'Bird', 'Reptile'))
    pet_event.breed = fake.random_element(elements=('Bulldog', 'Siamese', 'Parrot', 'Iguana'))
    pet_event.age = fake.random_int(min=1, max=20)
    pet_event.price = round(random.uniform(50.00, 5000.00), 2)
    pet_event.weight = round(random.uniform(1.0, 100.0), 2)
    pet_event.checkup_date = fake.date_this_decade().strftime('%Y-%m-%d')  # Convert date to string
    return pet_event

# Generate and send one event every 5 seconds
for i in range(1, 400):
    pet_event = generate_pet_shop_event()
    
    # Print the generated fake PetShopEvent (Protobuf object)
    print(f"Generated PetShopEvent: {pet_event}")
    
    # Send the event to Kafka topic
    producer.produce(topic=topic_name, key=pet_event.pet_id, value=pet_event, on_delivery=delivery_report)
    
    # Flush to ensure the message is sent
    producer.flush()

    # Wait before sending the next event
    time.sleep(5)


Generated PetShopEvent: pet_id: "30c1bab0-6382-43ac-9e17-48fdb6d6a97b"
pet_name: "Kristen"
species: "Dog"
breed: "Iguana"
age: 3
price: 4319.36
weight: 28.86
checkup_date: "2023-05-27"



ValueSerializationError: KafkaError{code=_VALUE_SERIALIZATION,val=-161,str="HTTPConnectionPool(host='schema-registry', port=8081): Max retries exceeded with url: /subjects/test-topic-value/versions?normalize=False (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x0000027FC76EB950>: Failed to resolve 'schema-registry' ([Errno 11001] getaddrinfo failed)"))"}